# TEXTUAL DATA ANALYSIS Exercise 6

## setup


In [1]:
!pip3 -q install datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.

### imports

In [2]:
from google.colab import userdata # for secret access
from datasets import load_dataset
from openai import OpenAI # for the connection
from pprint import pprint
from itertools import zip_longest
import pandas as pd
import time

### openai api key

In [3]:
api_key = userdata.get('openai')

### get some news texts

In [4]:
# use the exercise 1 material as a source

!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl

--2025-01-30 06:53:36--  http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3385882 (3.2M) [application/octet-stream]
Saving to: ‘news-en-2021.jsonl’

news-en-2021.jsonl  100%[===================>]   3.23M  2.72MB/s    in 1.2s    

2025-01-30 06:53:37 (2.72 MB/s) - ‘news-en-2021.jsonl’ saved [3385882/3385882]



In [5]:
!head -n 1 news-en-2021.jsonl

{"summary": "The decisions follow a meeting of government ministers at the House of the Estates on Thursday afternoon.", "tags": ["Kotimaan uutiset"], "text": "Finland's government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. \n \"There are still many open questions that need to be answered. At this point, it is impossible to promise that the pass will come or when it will come,\" Prime Minister  Sanna Marin  (SDP) told the media following the conclusion of the meeting. \n \"The government has given the green light to the Covid pass and preparations will continue,\" Marin added. \n Minister of Economic Affairs  Mika Lintilä  (Cen) told reporters immediately after the meeting that there was broad agreement between the coalition parties over the need for the certificate. \n \"It [the pass] is an important tool so that we will not need restrictions any more,\" Lintilä said. \n The gov

In [6]:
dataset = load_dataset('json', data_files='news-en-2021.jsonl')
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'tags', 'text', 'timestamp', 'title', 'url'],
        num_rows: 1059
    })
})


In [7]:
print(dataset['train'][100]['text'][:300]) # only 300 first

A feature allowing gamblers to limit their losses will be installed on slot machines operated by Finland's state gambling monopoly Veikkaus from September. 
 Loss limits allow players to place a cap on the amount of money they could potentially lose over a given period of time. 
 The move is the lat


In [8]:
articles = dataset['train'].select(range(10))
len(articles)

10

In [9]:
# ensure only len of 300 words

def get_words(text,n) -> str:
    words = text.split()
    return " ".join(words[:n])

In [10]:
fit_art = []

for a in articles:
  msg = get_words(a['text'],300)
  print(msg)
  print(f"count of words: {len(msg.split())}")
  fit_art.append(msg)

Finland's government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. "There are still many open questions that need to be answered. At this point, it is impossible to promise that the pass will come or when it will come," Prime Minister Sanna Marin (SDP) told the media following the conclusion of the meeting. "The government has given the green light to the Covid pass and preparations will continue," Marin added. Minister of Economic Affairs Mika Lintilä (Cen) told reporters immediately after the meeting that there was broad agreement between the coalition parties over the need for the certificate. "It [the pass] is an important tool so that we will not need restrictions any more," Lintilä said. The government also decided at Thursday afternoon's meeting to offer coronavirus vaccines to all 12- to 15-year-olds, starting as early as next week. "Fortunately, we have received an extra ba

In [11]:
fit_art[0]

'Finland\'s government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. "There are still many open questions that need to be answered. At this point, it is impossible to promise that the pass will come or when it will come," Prime Minister Sanna Marin (SDP) told the media following the conclusion of the meeting. "The government has given the green light to the Covid pass and preparations will continue," Marin added. Minister of Economic Affairs Mika Lintilä (Cen) told reporters immediately after the meeting that there was broad agreement between the coalition parties over the need for the certificate. "It [the pass] is an important tool so that we will not need restrictions any more," Lintilä said. The government also decided at Thursday afternoon\'s meeting to offer coronavirus vaccines to all 12- to 15-year-olds, starting as early as next week. "Fortunately, we have received an extra

### connect to OpenAI API

In [12]:
client = OpenAI(
    api_key=api_key
)

### TEST CALL

Write code to access the API, and retrieve results. Debug this with one short request until you know it works!

#### test with single word

In [13]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o-mini", # MUST BE 4o-mini >:(
)

In [14]:
print(chat_completion,end="\n\n")
print(chat_completion.choices, end="\n\n")
print(chat_completion.choices[0].message.content, end="\n\n")

ChatCompletion(id='chatcmpl-AvIii9MFhMYY2tNzY7P0LDBsO1tcf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test. How can I assist you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738220020, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=13, prompt_tokens=12, total_tokens=25, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test. How can I assist you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]

This is a test. How can I assist you fur

## TASKS

Write a prompt to extract named entities from a news article. Your prompt can either focus on one entity type (in this case, discard other types), or extract multiple entity types in the same prompt. Do not repeat the extraction separately for each entity type (let's save quota here!).

### algos

#### actual function to call the api

In [15]:
def api_call(message:str) -> str:
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"Make NER analysis using BIO tags, reply only *word:tag* and newline format: {message}",
          }
      ],
      model="gpt-4o-mini", # MUST BE 4o-mini >:(
  )
  return chat_completion.choices[0].message.content

#### harvest the NER tags

In [16]:
def harvester(message:str) -> list:
  NER_TAGS = []
  message = message.replace(" ", "") # we know this is the format inside the api_call function
  message = message.splitlines()
  for i,m in enumerate(message):
    ml = m.split(":")
    if len(ml) < 2:
      continue
    NER_TAGS.append((ml[0],ml[1]))
  return NER_TAGS


#### formatted printing

In [17]:
def formatted_printing(harvested_list:list) -> None:
  for row in harvested_list:
    print(f"{row[0]:<20} \t {row[1]}")

#### time it to var

In [18]:
def time_it_to_var(st:int, et:int) -> int:
  exe_time = et - st
  return exe_time

### TEST NER



In [19]:
st = time.time()

reply_message = api_call(fit_art[0])

et = time.time()

In [20]:
exe_time = time_it_to_var(st,et)
print(f'runtime of one call was {int(exe_time // 60)} minutes and {int(exe_time % 60)} seconds')

runtime of one call was 0 minutes and 23 seconds


In [21]:
harvested_tags = harvester(reply_message)

In [22]:
formatted_printing(harvested_tags)

Finland              	 B-LOC
's                   	 O
government           	 O
is                   	 O
pushing              	 O
ahead                	 O
with                 	 O
plans                	 O
to                   	 O
introduce            	 O
a                    	 O
Covid                	 B-MISC
pass                 	 O
following            	 O
a                    	 O
meeting              	 O
of                   	 O
ministers            	 O
at                   	 O
the                  	 O
House                	 B-ORG
of                   	 O
the                  	 O
Estates              	 I-ORG
in                   	 O
Helsinki             	 B-LOC
on                   	 O
Thursday             	 B-DATE
afternoon            	 O
.                    	 O
"There               	 O
are                  	 O
still                	 O
many                 	 O
open                 	 O
questions            	 O
that                 	 O
need                 	 O
to                   	 O

works...

Take 10 news articles from the same news data collection (Finnish or English), verify that the selected articles are not extremely long (should be less than 300 words each), discard or truncate longer documents.

In [23]:
# located inside fit_art variable

print(f'count of articles: {len(fit_art)}') # count of articles
for i,a in enumerate(fit_art):
  print(f'count of words in article {i+1}: {len(a.split())}') # count words

count of articles: 10
count of words in article 1: 300
count of words in article 2: 269
count of words in article 3: 226
count of words in article 4: 178
count of words in article 5: 300
count of words in article 6: 300
count of words in article 7: 300
count of words in article 8: 300
count of words in article 9: 300
count of words in article 10: 224


collect tags for all 10 articles

run time for one call was

In [24]:
print(f'runtime of one call was {int(exe_time // 60)} minutes and {int(exe_time % 60)} seconds')
print(f'so for {len(fit_art)} calls we can expect 10*exe_time')
ten_exe = 10 * exe_time
print(f'runtime for {len(fit_art)} calls estimated {int(ten_exe // 60)} minutes and {int(ten_exe % 60)} seconds', end="\n\n")


runtime of one call was 0 minutes and 23 seconds
so for 10 calls we can expect 10*exe_time
runtime for 10 calls estimated 3 minutes and 50 seconds



In [25]:
%%time

ner_tags_for_10_articles = {}

for i,a in enumerate(fit_art): # remember this is the truncated list
  reply = api_call(a) # call OpenAI API with the message
  harvested_tags = harvester(reply) # harvest the tags
  ner_tags_for_10_articles[f'article {str(i+1)}'] = harvested_tags

CPU times: user 1.33 s, sys: 226 ms, total: 1.55 s
Wall time: 5min 2s


In [26]:
its = []

for v in ner_tags_for_10_articles.values():
  its.append(v)


In [29]:
df = pd.DataFrame([x for x in ner_tags_for_10_articles.values()])
df = df.T
df.columns=[y for y in ner_tags_for_10_articles.keys()]

In [33]:
df.head(50)

,article 1,article 2,article 3,article 4,article 5,article 6,article 7,article 8,article 9,article 10
0,"(Finland, B-LOC)","(Rental, O)","(Double, O)","(The, O)","(A, O)","(Dozens, O)","(The, O)","(The, O)","(A, B-ORG)","(Helsinki, B-LOC)"
1,"('s, O)","(fees, O)","(Olympic, O)","(Regional, B-ORG)","(small, O)","(of, O)","(City, B-LOC)","(Union, ORG)","(gang, I-ORG)","(District, I-LOC)"
2,"(government, O)","(for, O)","(ice, O)","(State, I-ORG)","(percentage, O)","(coronavirus, B-ILL)","(of, I-LOC)","(of, O)","(of, I-ORG)","(Court, I-LOC)"
3,"(is, O)","(non-subsidised, O)","(hockey, O)","(Administrative, I-ORG)","(of, O)","(infections, I-ILL)","(Lahti, I-LOC)","(Upper, O)","(three, B-MISC)","(has, O)"
4,"(pushing, O)","(apartments, O)","(bronze, O)","(Agency, I-ORG)","(Finland, B-LOC)","(have, O)","(,, O)","(Secondary, O)","(masked, I-MISC)","(sentenced, O)"
5,"(ahead, O)","(rose, O)","(medallist, O)","(of, I-ORG)","('s, O)","(been, O)","(Europe, B-LOC)","(School, O)","(individuals, I-MISC)","(a, O)"
6,"(with, O)","(across, O)","(Emma, B-PER)","(Southern, B-LOC)","(healthcare, O)","(diagnosed, O)","('s, O)","(Students, ORG)","(have, O)","(43-year-old, O)"
7,"(plans, O)","(most, O)","(Terho, I-PER)","(Finland, I-LOC)","(workers, O)","(among, O)","(Green, B-MISC)","(has, O)","(stolen, O)","(taxi, O)"
8,"(to, O)","(of, O)","(has, O)","((Avi), B-ORG)","(are, O)","(a, O)","(Capital, I-MISC)","(issued, O)","(or, O)","(driver, O)"
9,"(introduce, O)","(Finland, LOC)","(been, O)","(has, O)","(yet, O)","(group, O)","(of, O)","(a, O)","(attempted, O)","(to, O)"
